# E1: Athlete won most golden medals each region in 2014 Summer Olympics
## - Mimic ROW_NUMBER() window function in PostgreSQL

In [1]:
import pandas as pd
import numpy as np

In [2]:
summer_games = pd.read_csv("summer_games.csv")
winter_games = pd.read_csv("winter_games.csv")
countries = pd.read_csv("countries.csv")
athletes = pd.read_csv("athletes.csv")
country_stats = pd.read_csv("country_stats.csv")

In [3]:
display(summer_games.head(2))
display(winter_games.head(2))
display(countries.head(2))
display(athletes.head(2))
display(country_stats.head(2))

,sport,event,year,athlete_id,country_id,bronze,silver,gold
0,Gymnastics,Gymnastics Men's Individual All-Around,2016-01-01,51,173,NaN,NaN,NaN
1,Gymnastics,Gymnastics Men's Floor Exercise,2016-01-01,51,173,NaN,NaN,NaN


,sport,event,year,athlete_id,country_id,bronze,silver,gold
0,Alpine Skiing,Alpine Skiing Women's Slalom,2014-01-01,126,89,NaN,NaN,NaN
1,Alpine Skiing,Alpine Skiing Women's Super G,2014-01-01,463,102,NaN,NaN,NaN


,id,country,region
0,1,AFG - Afghanistan,ASIA (EX. NEAR EAST)
1,2,ALB - Albania,EASTERN EUROPE


,id,name,gender,age,height,weight
0,51,Nstor Abad Sanjun,M,23,167.0,64.0
1,55,Antonio Abadia Beci,M,26,170.0,65.0


,year,country_id,gdp,population,nobel_prize_winners
0,2000-01-01,1,NaN,20093756.0,NaN
1,2000-01-01,2,3.632044e+09,3089027.0,NaN


In [4]:
# join all tables and filter out the ones whos gold column is null:
region_athlete_tb = \
athletes.merge(summer_games, left_on = "id", right_on = "athlete_id")\
        .merge(countries, left_on = 'country_id', right_on = "id")\
        .loc[lambda df: ~df['gold'].isna(), ['region', 'name','gold']]

In [5]:
region_athlete_tb.head()

,region,name,gold
13,WESTERN EUROPE,Ruth Beita Vila,1.0
16,WESTERN EUROPE,Mireia Belmonte Garca,1.0
123,NEAR EAST,Ruth Jebet,1.0
187,WESTERN EUROPE,"Nafissatou ""Nafi"" Thiam",1.0
263,WESTERN EUROPE,Fabian Hambchen,1.0


In [6]:
total_gold_group = region_athlete_tb.groupby(['region', 'name'])['gold'].sum().rename("total_golds").reset_index()

In [7]:
total_gold_group.head()

,region,name,total_golds
0,ASIA (EX. NEAR EAST),Joseph Isaac Schooling,1.0
1,ASIA (EX. NEAR EAST),Kenzo Shirai,1.0
2,ASIA (EX. NEAR EAST),Kohei Uchimura,2.0
3,ASIA (EX. NEAR EAST),Koji Yamamuro,1.0
4,ASIA (EX. NEAR EAST),Kosuke Hagino,1.0


In [9]:
# NOTE: here needs to use rank(method='first') to make ranking order like 1,2,3,4,5.... 
# (others such as 'min' and 'dense' would result in different order type)
total_gold_group.assign(rank = lambda df: df.groupby('region')['total_golds'].rank(method='first', ascending=False))\
                .loc[lambda df: df['rank'] == 1]

,region,name,total_golds,rank
2,ASIA (EX. NEAR EAST),Kohei Uchimura,2.0,1.0
12,C.W. OF IND. STATES,Aliya Farkhatovna Mustafina,1.0,1.0
17,EASTERN EUROPE,Katinka Hossz,3.0,1.0
30,LATIN AMER. & CARIB,Usain St. Leo Bolt,3.0,1.0
32,NEAR EAST,Ruth Jebet,1.0,1.0
80,NORTHERN AMERICA,"Michael Fred Phelps, II",5.0,1.0
98,OCEANIA,Brittany Joyce Elmslie,1.0,1.0
105,SUB-SAHARAN AFRICA,Almaz Ayana Eba,1.0,1.0
121,WESTERN EUROPE,Max Antony Whitlock,2.0,1.0
